In [2]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import librosa
from sklearn.model_selection import train_test_split

from src.preprocess import transform_to_data_loader
from src.train import train_model
from src.preprocess_utils import create_binary_labels

from torchinfo import summary
from torchaudio.transforms import MFCC

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
X, y = np.load("augmented_data.npy"), np.load("augmented_labels.npy")

In [ ]:
# Binary task with transformation

transform = MFCC(sample_rate=16000, n_mfcc=20, melkwargs={"n_fft": 400, "hop_length": 40, "n_mels": 20, "center": False})
X_tensor = torch.from_numpy(X)

indices = [i for i in range(0, X_tensor.shape[0], 5000)]
indices.append(X_tensor.shape[0])

X_transformed = transform(X_tensor[indices[0]:indices[1]])
for i in range(1, len(indices)-1):
    X_transformed = np.concatenate((X_transformed, transform(X_tensor[indices[i]:indices[i+1]]).numpy()), 0)

X_transformed_bin = X_transformed
y_bin = create_binary_labels(y)

X_train, X_test, y_train, y_test = train_test_split(
    X_transformed_bin, y_bin, test_size=0.2, random_state=222
)

del X_transformed_bin, y_bin

In [ ]:
class bin_Transformer(nn.Module):
    
    def __init__(self, d_model, n_head, num_layers):
        super().__init__()
        self.trans_enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, batch_first=True, activation="relu").to(device)
        self.transformer_encoder = nn.TransformerEncoder(self.trans_enc_layer, num_layers=num_layers).to(device)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(d_model, 64)
        self.fc2 = nn.Linear(64, 1)
        self.bc1 = nn.BatchNorm1d(d_model)
        self.bc2 = nn.BatchNorm1d(64)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)

    def forward(self, x):
        out = self.conv(x.unsqueeze(1)).squeeze(1)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.transformer_encoder(out)[:, -1, :]
        out = self.dropout(out)
        out = self.bc1(out)
        out = self.fc1(out)
        out = self.dropout(out)
        out = self.relu(out)
        out = self.bc2(out)
        out = self.fc2(out)
        out = self.dropout(out)
        out = self.sigmoid(out)
        return out